In [1]:
import sys
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

In [3]:
train_features = pd.read_csv('train_features.csv')
train_targets_scored = pd.read_csv('train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('train_targets_nonscored.csv')

test_features = pd.read_csv('test_features.csv')
sample_submission = pd.read_csv('sample_submission.csv')


In [4]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [5]:
def seed_everything(seed=1903):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=1903)

In [6]:
# GENES
n_comp = 28

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
data2 = (PCA(n_components=n_comp, random_state=1903).fit_transform(data[GENES]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)
print("train_feature.shape:", train_features.shape,"test_features.shape",test_features.shape)

#CELLS
n_comp = 5

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
data2 = (PCA(n_components=n_comp, random_state=1903).fit_transform(data[CELLS]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)
print("train_feature.shape:", train_features.shape,"test_features.shape",test_features.shape)

train_feature.shape: (23814, 904) test_features.shape (3982, 904)
train_feature.shape: (23814, 909) test_features.shape (3982, 909)


In [7]:
from sklearn.feature_selection import VarianceThreshold


var_thresh = VarianceThreshold(threshold=0.4)
data = train_features.append(test_features)

feature_cols = train_features.columns.values[4:]
data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

train_features_transformed = data_transformed[ : train_features.shape[0]]
test_features_transformed = data_transformed[-test_features.shape[0] : ]
print("data_transformed.shape",data_transformed.shape)


data_transformed.shape (27796, 901)


In [10]:
train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed, columns=feature_cols[var_thresh.get_support()])], axis=1)


test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed, columns=feature_cols[var_thresh.get_support()])], axis=1)

train_features

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,pca_G-23,pca_G-24,pca_G-25,pca_G-26,pca_G-27,pca_C-0,pca_C-1,pca_C-2,pca_C-3,pca_C-4
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.743672,0.971888,0.756497,-1.093429,0.749532,-7.285008,0.608401,-0.007462,0.186788,-0.760827
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,0.605557,-0.905093,-0.756600,1.694579,3.023896,-7.417466,-0.756903,0.139662,-0.751076,-0.675731
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,1.597964,-1.571965,-1.768986,-1.278462,-2.976800,-2.247580,0.226875,0.088391,0.969065,-0.169469
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,3.732154,-1.548930,-1.454501,-0.594539,-2.130195,13.943314,7.308940,0.512217,-3.353907,-1.337379
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,-1.322954,0.293573,-0.416163,1.286734,2.086231,-6.307752,0.262640,0.266405,0.634856,-0.196170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,trt_cp,24,D2,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,...,-0.290860,-2.968704,0.421827,-0.843860,0.304833,-6.398104,0.103700,0.055015,-0.729866,-0.093763
23810,id_fffb70c0c,trt_cp,24,D2,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,...,-2.336899,0.627563,0.446241,-0.224986,-0.421141,-4.029873,-1.008948,1.117319,0.148991,-0.005460
23811,id_fffc1c3f4,ctl_vehicle,48,D2,0.3942,0.3756,0.3109,-0.7389,0.5505,-0.0159,...,0.360302,0.001423,0.345062,-0.530562,-2.058057,-8.302736,0.451241,0.643414,-0.618859,0.647034
23812,id_fffcb9e7c,trt_cp,24,D1,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,...,-1.854534,-1.225535,1.419350,-5.347429,-0.783404,-7.838803,0.176862,0.256411,0.313841,0.417585


In [13]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

target = train[train_targets_scored.columns]

train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)
train.head(2)

,sig_id,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,...,0,0,0,0,0,0,0,0,0,0


In [14]:
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

In [15]:
folds = train.copy()

mskf = MultilabelStratifiedKFold(n_splits=7)

for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
    folds.loc[v_idx, 'kfold'] = int(f)

folds['kfold'] = folds['kfold'].astype(int)

In [16]:
def process_data(data):
    
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
#     data.loc[:, 'cp_time'] = data.loc[:, 'cp_time'].map({24: 0, 48: 1, 72: 2})
#     data.loc[:, 'cp_dose'] = data.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})

# --------------------- Normalize ---------------------
#     for col in GENES:
#         data[col] = (data[col]-np.mean(data[col])) / (np.std(data[col]))
    
#     for col in CELLS:
#         data[col] = (data[col]-np.mean(data[col])) / (np.std(data[col]))
    
#--------------------- Removing Skewness ---------------------
#     for col in GENES + CELLS:
#         if(abs(data[col].skew()) > 0.75):
            
#             if(data[col].skew() < 0): # neg-skewness
#                 data[col] = data[col].max() - data[col] + 1
#                 data[col] = np.sqrt(data[col])
            
#             else:
#                 data[col] = np.sqrt(data[col])
    
    return data

In [17]:
feature_cols = [c for c in process_data(folds).columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id']]
len(feature_cols)

906

In [18]:
gen_cols = [c for c in feature_cols if 'g-' == c[:2]]
cell_cols = [c for c in feature_cols if 'c-' == c[:2]]
pca_gen_cols = [c for c in feature_cols if 'pca_G' == c[:5]]
pca_cell_cols = [c for c in feature_cols if 'pca_C' == c[:5]]
ohe_cols = [c for c in feature_cols if c not in gen_cols+cell_cols+pca_gen_cols+pca_cell_cols]
print(len(gen_cols), len(cell_cols), len(pca_gen_cols), len(pca_cell_cols),len(ohe_cols))

768 100 28 5 5


In [19]:
class Model(nn.Module):
    def __init__(self, num_gen_features, num_cell_features, num_genpca_features, num_cellpca_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.gen_batch_norm1 = nn.BatchNorm1d(num_gen_features)
        self.gen_dropout1 = nn.Dropout(0.2)
        self.gen_dense1 = nn.utils.weight_norm(nn.Linear(num_gen_features, hidden_size))
        self.gen_batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.gen_dropout2 = nn.Dropout(0.3)
        self.gen_dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.cell_batch_norm1 = nn.BatchNorm1d(num_cell_features)
        self.cell_dropout1 = nn.Dropout(0.2)
        self.cell_dense1 = nn.utils.weight_norm(nn.Linear(num_cell_features, int(hidden_size/2)))
        self.cell_batch_norm2 = nn.BatchNorm1d(int(hidden_size/2))
        self.cell_dropout2 = nn.Dropout(0.3)
        self.cell_dense2 = nn.utils.weight_norm(nn.Linear(int(hidden_size/2), int(hidden_size/2)))
        
        self.genpca_batch_norm1 = nn.BatchNorm1d(num_genpca_features)
        self.genpca_dropout1 = nn.Dropout(0.2)
        self.genpca_dense1 = nn.utils.weight_norm(nn.Linear(num_genpca_features, num_genpca_features))
        self.genpca_batch_norm2 = nn.BatchNorm1d(num_genpca_features)
        self.genpca_dropout2 = nn.Dropout(0.3)
        self.genpca_dense2 = nn.utils.weight_norm(nn.Linear(num_genpca_features, num_genpca_features))
        
        self.cellpca_batch_norm1 = nn.BatchNorm1d(num_cellpca_features)
        self.cellpca_dropout1 = nn.Dropout(0.2)
        self.cellpca_dense1 = nn.utils.weight_norm(nn.Linear(num_cellpca_features, num_cellpca_features))
        self.cellpca_batch_norm2 = nn.BatchNorm1d(num_cellpca_features)
        self.cellpca_dropout2 = nn.Dropout(0.3)
        self.cellpca_dense2 = nn.utils.weight_norm(nn.Linear(num_cellpca_features, num_cellpca_features))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size + int(hidden_size/2) + num_genpca_features + num_cellpca_features)
        self.dropout3 = nn.Dropout(0.25)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size + int(hidden_size/2) + num_genpca_features + num_cellpca_features, num_targets))
        
        
    
    def forward(self, gen_x, cell_x, genpca_x, cellpca_x):
#         print(cell_x.shape)
        gen_x = self.gen_batch_norm1(gen_x)
        gen_x = self.gen_dropout1(gen_x)
        gen_x = F.relu(self.gen_dense1(gen_x))
        gen_x = self.gen_batch_norm2(gen_x)
        gen_x = self.gen_dropout2(gen_x)
        gen_x = F.relu(self.gen_dense2(gen_x))
        
        cell_x = self.cell_batch_norm1(cell_x)
        cell_x = self.cell_dropout1(cell_x)
        cell_x = F.relu(self.cell_dense1(cell_x))
        cell_x = self.cell_batch_norm2(cell_x)
        cell_x = self.cell_dropout2(cell_x)
        cell_x = F.relu(self.cell_dense2(cell_x))
        
        genpca_x = self.genpca_batch_norm1(genpca_x)
        genpca_x = self.genpca_dropout1(genpca_x)
        genpca_x = F.relu(self.genpca_dense1(genpca_x))
        genpca_x = self.genpca_batch_norm2(genpca_x)
        genpca_x = self.genpca_dropout2(genpca_x)
        genpca_x = F.relu(self.genpca_dense2(genpca_x))
        
        cellpca_x = self.cellpca_batch_norm1(cellpca_x)
        cellpca_x = self.cellpca_dropout1(cellpca_x)
        cellpca_x = F.relu(self.cellpca_dense1(cellpca_x))
        cellpca_x = self.cellpca_batch_norm2(cellpca_x)
        cellpca_x = self.cellpca_dropout2(cellpca_x)
        cellpca_x = F.relu(self.cellpca_dense2(cellpca_x))
                
        x = torch.cat((gen_x, cell_x, genpca_x, cellpca_x), dim=1)
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        return x

In [15]:
class MoADataset:
    def __init__(self, gen_features, cell_features, genpca_features, cellpca_features, targets):
        self.gen_features = gen_features
        self.cell_features = cell_features
        self.genpca_features = genpca_features
        self.cellpca_features = cellpca_features
        self.targets = targets
        
    def __len__(self):
        return (self.gen_features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'gen_x' : torch.tensor(self.gen_features[idx, :], dtype=torch.float),
            'cell_x' : torch.tensor(self.cell_features[idx, :], dtype=torch.float),
            'genpca_x' : torch.tensor(self.genpca_features[idx, :], dtype=torch.float),
            'cellpca_x' : torch.tensor(self.cellpca_features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, gen_features, cell_features, genpca_features, cellpca_features):
        self.gen_features = gen_features
        self.cell_features = cell_features
        self.genpca_features = genpca_features
        self.cellpca_features = cellpca_features
        
    def __len__(self):
        return (self.gen_features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'gen_x' : torch.tensor(self.gen_features[idx, :], dtype=torch.float),
            'cell_x' : torch.tensor(self.cell_features[idx, :], dtype=torch.float),  
            'genpca_x' : torch.tensor(self.genpca_features[idx, :], dtype=torch.float),
            'cellpca_x' : torch.tensor(self.cellpca_features[idx, :], dtype=torch.float)  
        }
        return dct


In [16]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        gen_x, cell_x, genpca_x, cellpca_x, targets = data['gen_x'].to(device), data['cell_x'].to(device), data['genpca_x'].to(device), data['cellpca_x'].to(device), data['y'].to(device)
        outputs = model(gen_x, cell_x, genpca_x, cellpca_x)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        gen_x, cell_x, genpca_x, cellpca_x, targets = data['gen_x'].to(device), data['cell_x'].to(device), data['genpca_x'].to(device), data['cellpca_x'].to(device), data['y'].to(device)
        outputs = model(gen_x, cell_x, genpca_x, cellpca_x)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        gen_x, cell_x, genpca_x, cellpca_x = data['gen_x'].to(device), data['cell_x'].to(device), data['genpca_x'].to(device), data['cellpca_x'].to(device)

        with torch.no_grad():
            outputs = model(gen_x, cell_x, genpca_x, cellpca_x)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds
   

In [17]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)
EPOCHS = 25
BATCH_SIZE = 1024
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 7
EARLY_STOPPING_STEPS = 10
EARLY_STOP = True

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=1024

cuda


In [18]:
import time
def run_training(fold, seed):
    
    seed_everything(seed)
    
    train = process_data(folds)
    test_ = process_data(test)
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index

    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)

    gen_x_train, cell_x_train, genpca_x_train, cellpca_x_train, y_train = train_df[gen_cols + ohe_cols].values, train_df[cell_cols + ohe_cols].values, train_df[pca_gen_cols + ohe_cols].values, train_df[pca_cell_cols + ohe_cols].values, train_df[target_cols].values
    gen_x_valid, cell_x_valid, genpca_x_valid, cellpca_x_valid, y_valid = valid_df[gen_cols + ohe_cols].values, valid_df[cell_cols + ohe_cols].values, valid_df[pca_gen_cols + ohe_cols].values, valid_df[pca_cell_cols + ohe_cols].values, valid_df[target_cols].values
    #     print("len target_cols", len(target_cols), train_df[target_cols].shape, train_df[target_cols].reset_index(drop=True).shape)
    #     print("shapes", x_train.shape, y_train.shape, x_valid.shape, y_valid.shape)

    train_dataset = MoADataset(gen_x_train, cell_x_train, genpca_x_train, cellpca_x_train, y_train)
    valid_dataset = MoADataset(gen_x_valid, cell_x_valid, genpca_x_valid, cellpca_x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
#     print('# of target cols', len(target_cols))
    
    model = Model(
        num_gen_features=len(gen_cols + ohe_cols),
        num_cell_features=len(cell_cols + ohe_cols),
        num_genpca_features=len(pca_gen_cols + ohe_cols),
        num_cellpca_features=len(pca_cell_cols + ohe_cols),
        num_targets=len(target_cols),
        hidden_size=hidden_size
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.BCEWithLogitsLoss()
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        start = time.time()
        train_loss = train_fn(model, optimizer,scheduler, loss_fn, trainloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
        valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        elapse = time.time() - start
        print(f"FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss} {elapse:.2f} seconds")
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"FOLD{fold}_.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            
    
    #--------------------- PREDICTION---------------------
    gen_x_test, cell_x_test, genpca_x_test, cellpca_x_test = test_[gen_cols + ohe_cols].values, test_[cell_cols + ohe_cols].values, test_[pca_gen_cols + ohe_cols].values, test_[pca_cell_cols + ohe_cols].values
    testdataset = TestDataset(gen_x_test, cell_x_test, genpca_x_test, cellpca_x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_gen_features=len(gen_cols + ohe_cols),
        num_cell_features=len(cell_cols + ohe_cols),
        num_genpca_features=len(pca_gen_cols + ohe_cols),
        num_cellpca_features=len(pca_cell_cols + ohe_cols),
        num_targets=len(target_cols),
        hidden_size=hidden_size
    )
    
    model.load_state_dict(torch.load(f"FOLD{fold}_.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions

In [19]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        oof_, pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [20]:
# class Model(nn.Module):
#     def __init__(self, num_gen_features, num_cell_features,num_targets, hidden_size):
#         super(Model, self).__init__()
#         self.gen_batch_norm1 = nn.BatchNorm1d(num_gen_features)
#         self.gen_dropout1 = nn.Dropout(0.2)
#         self.gen_dense1 = nn.utils.weight_norm(nn.Linear(num_gen_features, hidden_size))
#         self.gen_batch_norm2 = nn.BatchNorm1d(hidden_size)
#         self.gen_dropout2 = nn.Dropout(0.3)
#         self.gen_dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
#         self.cell_batch_norm1 = nn.BatchNorm1d(num_cell_features)
#         self.cell_dropout1 = nn.Dropout(0.2)
#         self.cell_dense1 = nn.utils.weight_norm(nn.Linear(num_cell_features, int(hidden_size/2)))
#         self.cell_batch_norm2 = nn.BatchNorm1d(int(hidden_size/2))
#         self.cell_dropout2 = nn.Dropout(0.3)
#         self.cell_dense2 = nn.utils.weight_norm(nn.Linear(int(hidden_size/2), int(hidden_size/2)))
        
#         self.batch_norm3 = nn.BatchNorm1d(hidden_size + int(hidden_size/2) )
#         self.dropout3 = nn.Dropout(0.25)
#         self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size + int(hidden_size/2), num_targets))
        
        
    
#     def forward(self, gen_x, cell_x):
# #         print(cell_x.shape)
#         gen_x = self.gen_batch_norm1(gen_x)
#         gen_x = self.gen_dropout1(gen_x)
#         gen_x = F.relu(self.gen_dense1(gen_x))
#         gen_x = self.gen_batch_norm2(gen_x)
#         gen_x = self.gen_dropout2(gen_x)
#         gen_x = F.relu(self.gen_dense2(gen_x))

#         cell_x = self.cell_batch_norm1(cell_x)
#         cell_x = self.cell_dropout1(cell_x)
#         cell_x = F.relu(self.cell_dense1(cell_x))
#         cell_x = self.cell_batch_norm2(cell_x)
#         cell_x = self.cell_dropout2(cell_x)
#         cell_x = F.relu(self.cell_dense2(cell_x))
                
#         x = torch.cat((gen_x,cell_x),dim=1)
#         x = self.batch_norm3(x)
#         x = self.dropout3(x)
#         x = self.dense3(x)
#         return x

In [21]:
# Averaging on multiple SEEDS

SEED = [1903, 1881]
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_cols] = oof
test[target_cols] = predictions

FOLD: 0, EPOCH: 0, train_loss: 0.47592380550392227
FOLD: 0, EPOCH: 0, valid_loss: 0.023442984223365784 3.70 seconds
FOLD: 0, EPOCH: 1, train_loss: 0.0208129358484226
FOLD: 0, EPOCH: 1, valid_loss: 0.018651681020855904 2.99 seconds
FOLD: 0, EPOCH: 2, train_loss: 0.018767618726254726
FOLD: 0, EPOCH: 2, valid_loss: 0.01792339824140072 3.11 seconds
FOLD: 0, EPOCH: 3, train_loss: 0.01777006941689115
FOLD: 0, EPOCH: 3, valid_loss: 0.017474866695702074 3.12 seconds
FOLD: 0, EPOCH: 4, train_loss: 0.017480886889761
FOLD: 0, EPOCH: 4, valid_loss: 0.017412602864205838 3.01 seconds
FOLD: 0, EPOCH: 5, train_loss: 0.017356891305420268
FOLD: 0, EPOCH: 5, valid_loss: 0.01748192425817251 3.11 seconds
FOLD: 0, EPOCH: 6, train_loss: 0.017375252374327507
FOLD: 0, EPOCH: 6, valid_loss: 0.017758790627121924 3.07 seconds
FOLD: 0, EPOCH: 7, train_loss: 0.01740945389076155
FOLD: 0, EPOCH: 7, valid_loss: 0.017338837757706643 2.91 seconds
FOLD: 0, EPOCH: 8, train_loss: 0.017430662659003216
FOLD: 0, EPOCH: 8, val

In [22]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)


y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

score = 0
for i in range(len(target_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / target.shape[1]
    
print("CV log_loss: ", score)

CV log_loss:  0.014745746816591205


In [23]:
#sub = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
#sub.to_csv('submission.csv', index=False)